In [2]:
# ==============================================================================
# 1. SETUP THE ENVIRONMENT
# ==============================================================================
print("🚀 Starting setup...")

# Install Flask (the web server) and pyngrok (the tunneling tool)
!pip install -q flask pyngrok google-generativeai pandas

import os
import threading
import time
from pyngrok import ngrok
from flask import Flask, request, jsonify, render_template
import google.generativeai as genai
import pandas as pd
import json
import io
from google.colab import userdata

print("✅ Setup complete.")

🚀 Starting setup...
✅ Setup complete.


In [3]:
# ==============================================================================
# 2. FETCH SECRET KEYS
# ==============================================================================
print("🔑 Fetching API keys from Colab Secrets...")

try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')

    # Configure the Google AI client once, globally.
    genai.configure(api_key=GOOGLE_API_KEY)
    print("✅ Google API Key configured.")

    # Set the ngrok authtoken.
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    print("✅ ngrok Authtoken configured.")

except Exception as e:
    print(f"🛑 ERROR: Could not get secret key. Please make sure GOOGLE_API_KEY and NGROK_AUTH_TOKEN are set correctly in Colab Secrets (View > Secrets).")
    # Stop execution if keys are not found
    raise e

🔑 Fetching API keys from Colab Secrets...
✅ Google API Key configured.
✅ ngrok Authtoken configured.


In [4]:
# ==============================================================================
# 3. CREATE THE HTML FILE FOR THE FRONTEND
# ==============================================================================
print("📝 Creating HTML file for the web app interface...")

# Create the 'templates' directory for our HTML file
os.makedirs("templates", exist_ok=True)

# --- This is your templates/index.html file ---
index_html_code = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8"><meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>AI Data Agent</title>
    <script src="https://cdn.tailwindcss.com"></script><script src="https://cdn.jsdelivr.net/npm/chart.js"></script><script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
    <style> body { font-family: sans-serif; } #result-container table { width: 100%; border-collapse: collapse; } #result-container th, #result-container td { border: 1px solid #ddd; padding: 8px; } </style>
</head>
<body class="bg-gray-100 flex items-center justify-center min-h-screen p-4">
    <div class="w-full max-w-3xl bg-white rounded-lg shadow-xl p-8">
        <div class="text-center mb-8"><h1 class="text-4xl font-bold text-gray-800">AI Data Agent</h1><p class="text-gray-600 mt-2">The Real Web App, Launched from Colab</p></div>
        <form id="analysis-form" class="space-y-6">
            <div><label class="block text-lg font-medium text-gray-700">Step 1: Upload CSV</label><input id="csv-file-input" name="file" type="file" class="mt-2 block w-full text-sm text-gray-500 file:mr-4 file:py-2 file:px-4 file:rounded-full file:border-0 file:text-sm file:font-semibold file:bg-violet-50 file:text-violet-700 hover:file:bg-violet-100" accept=".csv" required></div>
            <div><label for="prompt" class="block text-lg font-medium text-gray-700">Step 2: Ask a Question</label><textarea id="prompt" name="prompt" rows="4" class="mt-2 shadow-sm block w-full sm:text-sm border-gray-300 rounded-md p-3" placeholder="e.g., 'Create a bar chart of sales by region.'" required></textarea></div>
            <div><button id="analyze-button" type="submit" class="w-full flex justify-center py-3 px-4 border border-transparent rounded-md shadow-sm text-sm font-medium text-white bg-indigo-600 hover:bg-indigo-700">Analyze & Visualize</button></div>
        </form>
        <div id="result-section" class="hidden mt-6"><h2 class="text-lg font-medium text-gray-700 mb-2">Analysis Results</h2><div id="loader" class="hidden text-center my-4"><p class="text-gray-600 mt-2">The AI is thinking...</p></div><div id="result-container" class="bg-gray-50 p-4 rounded-md border"></div><div id="chart-container" class="mt-4"><canvas id="myChart"></canvas></div></div>
    </div>
    <script>
        document.getElementById('analysis-form').addEventListener('submit', async (e) => {
            e.preventDefault();
            const form = e.target;
            const loader = document.getElementById('loader');
            const resultSection = document.getElementById('result-section');
            const resultContainer = document.getElementById('result-container');
            const chartContainer = document.getElementById('chart-container');
            let chartInstance = Chart.getChart("myChart");
            if(chartInstance) { chartInstance.destroy(); }

            loader.classList.remove('hidden');
            resultSection.classList.remove('hidden');
            resultContainer.innerHTML = '';
            chartContainer.style.display = 'none';

            try {
                const response = await fetch('/analyze', { method: 'POST', body: new FormData(form) });
                if (!response.ok) throw new Error((await response.json()).error || 'Server error');
                const data = await response.json();
                if (data.text_response) resultContainer.innerHTML = marked.parse(data.text_response);
                if (data.chart_config) {
                    chartContainer.style.display = 'block';
                    new Chart(document.getElementById('myChart'), data.chart_config);
                }
            } catch (error) { resultContainer.innerHTML = `<p class="text-red-500">Error: ${error.message}</p>`; }
            finally { loader.classList.add('hidden'); }
        });
    </script>
</body></html>
"""
with open("templates/index.html", "w") as f:
    f.write(index_html_code)

print("✅ HTML file created.")

📝 Creating HTML file for the web app interface...
✅ HTML file created.


In [5]:
# ==============================================================================
# 4. DEFINE AND LAUNCH THE WEB SERVER
# ==============================================================================
print("🚀 Launching web application...")

# --- CLEANUP PREVIOUS RUNS ---
print("🧹 Cleaning up any previous server instances...")
ngrok.kill()
!fuser -k 5000/tcp > /dev/null 2>&1
time.sleep(2)
print("✅ Cleanup complete.")

# --- DEFINE THE FLASK APP ---
app = Flask(__name__, template_folder='templates')

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/analyze', methods=['POST'])
def analyze():
    if 'file' not in request.files: return jsonify({"error": "No file part"}), 400
    file = request.files['file']
    user_question = request.form.get('prompt', '')
    if file.filename == '' or not user_question: return jsonify({"error": "Missing file or prompt"}), 400

    try:
        df = pd.read_csv(io.BytesIO(file.read()))
        model = genai.GenerativeModel('gemini-1.5-flash-latest')
        data_sample = df.head(100).to_csv(index=False)
        headers = ", ".join(df.columns)

        prompt = f'''
            You are a data scientist. Analyze the CSV data with headers: {headers}.
            Data sample: {data_sample}
            User's question: "{user_question}"
            INSTRUCTIONS: Provide a Markdown text answer. If a chart is requested, YOU MUST provide a 'json' code block with a valid Chart.js config.
            JSON STRICT MODE: ALL keys and string values MUST be in DOUBLE QUOTES. Do not use trailing commas.
        '''
        response = model.generate_content(prompt)
        raw_text = response.text
        text_part = raw_text
        chart_json = None
        json_match_start = raw_text.find("```json")
        if json_match_start != -1:
            text_part = raw_text[:json_match_start].strip()
            json_match_end = raw_text.rfind("```")
            json_block = raw_text[json_match_start + 7 : json_match_end].strip()
            chart_json = json.loads(json_block)

        return jsonify({"text_response": text_part, "chart_config": chart_json})
    except Exception as e:
        print(f"--- ERROR IN /analyze ROUTE --- \n{e}")
        return jsonify({"error": str(e)}), 500

# --- RUN THE APP AND CREATE THE PUBLIC URL ---
public_url = ngrok.connect(5000)
print("🎉 Your web app is live!")
print(f"👉 Click here to access it: {public_url}")

# Run the Flask app
# This will run forever until you manually stop the cell
app.run(port=5000)

🚀 Launching web application...
🧹 Cleaning up any previous server instances...
✅ Cleanup complete.
🎉 Your web app is live!
👉 Click here to access it: NgrokTunnel: "https://77f05f541bbd.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [11/Jul/2025 22:18:01] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jul/2025 22:18:03] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [11/Jul/2025 22:18:46] "POST /analyze HTTP/1.1" 200 -
